In [9]:
# ============================================================================
# BÖLÜM 1: KÜTÜPHANE YÜKLEMELERİ VE KURULUM
# ============================================================================

print("📦 Gerekli kütüphaneler yükleniyor...")

# Temel kütüphaneler
import os
from dotenv import load_dotenv
import google.generativeai as genai

# PDF işleme
from PyPDF2 import PdfReader

# Text processing
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Vector database
import chromadb
from chromadb.config import Settings

from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

# Utilities
import numpy as np
from tqdm import tqdm
import json

print("✅ Kütüphaneler yüklendi!")

📦 Gerekli kütüphaneler yükleniyor...
✅ Kütüphaneler yüklendi!


In [4]:
# ============================================================================
# BÖLÜM 2: ÇEVRE DEĞİŞKENLERİ VE API AYARLARI
# ============================================================================

print("\n🔑 API anahtarı yükleniyor...")

# .env dosyasından API key'i yükle
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

if not GEMINI_API_KEY:
    raise ValueError("❌ GEMINI_API_KEY bulunamadı! .env dosyanızı kontrol edin.")

# Gemini API'yi yapılandır
genai.configure(api_key=GEMINI_API_KEY)

print("✅ API anahtarı yüklendi!")
print(f"🔐 API Key (ilk 10 karakter): {GEMINI_API_KEY[:10]}...")


🔑 API anahtarı yükleniyor...
✅ API anahtarı yüklendi!
🔐 API Key (ilk 10 karakter): AIzaSyDGYP...


In [5]:
# ============================================================================
# BÖLÜM 3: PDF OKUMA VE METİN ÇIKARMA
# ============================================================================

print("\n📄 PDF belgesi okunuyor...")

PDF_PATH = "../Data/2024-2028-İstanbul-bölge-planı-taslak.pdf"

def extract_text_from_pdf(pdf_path):
    """PDF dosyasından metin çıkarır"""
    try:
        reader = PdfReader(pdf_path)
        text = ""
        
        print(f"📖 Toplam sayfa sayısı: {len(reader.pages)}")
        
        for i, page in enumerate(tqdm(reader.pages, desc="Sayfalar okunuyor")):
            text += page.extract_text()
        
        return text
    except Exception as e:
        print(f"❌ PDF okuma hatası: {e}")
        return None

# PDF'i oku
raw_text = extract_text_from_pdf(PDF_PATH)

if raw_text:
    print(f"\n✅ PDF başarıyla okundu!")
    print(f"📊 Toplam karakter sayısı: {len(raw_text):,}")
    print(f"📊 Toplam kelime sayısı: {len(raw_text.split()):,}")
    print(f"\n📝 İlk 500 karakter:\n{raw_text[:500]}")
else:
    raise ValueError("❌ PDF okunamadı!")


📄 PDF belgesi okunuyor...
📖 Toplam sayfa sayısı: 259


Sayfalar okunuyor: 100%|██████████| 259/259 [00:19<00:00, 13.38it/s]


✅ PDF başarıyla okundu!
📊 Toplam karakter sayısı: 714,440
📊 Toplam kelime sayısı: 86,220

📝 İlk 500 karakter:
0 
    
 1 TAKDİM  
 
Ülkemizin  sosyo -ekonomik  gelişmişlik  seviyesini  topyekûn  artırma  hedefi  doğrultusunda  Sayın  
Cumhurbaşkanımızın  liderliğinde  Millî  Teknoloji  Hamlesi  ile yüksek  teknolojili  ve katma  değerli  üretimi  
destekliyoruz.  Küresel  rekabet  gücümüzü  yükseltecek,  ekonomik  ve teknolojik  bağımsızlığımızı  
güçlendirecek  adımlar  atıyoruz.  Diğer  yandan  da Yerel  Kalkınma  Hamlesi  ile 81 ilimizin  potansiyelini  ve 
yetkinliklerini  harekete  geçirecek  polit


In [6]:
# ============================================================================
# BÖLÜM 4: METİN BÖLÜMLEME (CHUNKING)
# ============================================================================

print("\n✂️ Metin parçalara bölünüyor...")

# RecursiveCharacterTextSplitter ile metni böl
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,        # Her chunk 1000 karakter
    chunk_overlap=200,      # Chunk'lar arası 200 karakter örtüşme
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

chunks = text_splitter.split_text(raw_text)

print(f"✅ Metin bölümlendi!")
print(f"📊 Toplam chunk sayısı: {len(chunks)}")
print(f"📊 Ortalama chunk uzunluğu: {sum(len(c) for c in chunks) // len(chunks)} karakter")
print(f"\n📝 İlk chunk örneği:\n{chunks[0]}")


✂️ Metin parçalara bölünüyor...
✅ Metin bölümlendi!
📊 Toplam chunk sayısı: 875
📊 Ortalama chunk uzunluğu: 948 karakter

📝 İlk chunk örneği:
0 
    
 1 TAKDİM  
 
Ülkemizin  sosyo -ekonomik  gelişmişlik  seviyesini  topyekûn  artırma  hedefi  doğrultusunda  Sayın  
Cumhurbaşkanımızın  liderliğinde  Millî  Teknoloji  Hamlesi  ile yüksek  teknolojili  ve katma  değerli  üretimi  
destekliyoruz.  Küresel  rekabet  gücümüzü  yükseltecek,  ekonomik  ve teknolojik  bağımsızlığımızı  
güçlendirecek  adımlar  atıyoruz.  Diğer  yandan  da Yerel  Kalkınma  Hamlesi  ile 81 ilimizin  potansiyelini  ve 
yetkinliklerini  harekete  geçirecek  politikalarla  birlikte  tüm bölgelerimizin  bu yolculuğa  azami  katkıyı  
sunmasına  yönelik  program  ve projeleri  hayata  geçiriyoruz.  
Küresel  değer  zincirleriyle  ilişkiler,  üretim  yapısı,  dış ticaret  düzeyi,  girişimcilik  kapasitesi,  Ar-Ge ve yenilik  
performansı  ve diğer  illerle  sosyo -ekonomik  ilişkiler  açısından  illerimiz  ayrışmaktadı

In [12]:
# ============================================================================
# BÖLÜM 5: EMBEDDİNG OLUŞTURMA
# ============================================================================

print("\n🧠 Embedding'ler oluşturuluyor...")

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


print("✅ Embedding modeli hazır!")
print(f"📦 Model: embedding-001")

# Test embedding
test_embedding = embedding_model.embed_query("İstanbul bölge planı")
print(f"📊 Embedding boyutu: {len(test_embedding)}")


🧠 Embedding'ler oluşturuluyor...



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\ozanm\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ozanm\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Embedding modeli hazır!
📦 Model: embedding-001
📊 Embedding boyutu: 384


In [13]:
print("\n💾 Vector database oluşturuluyor...")

# ChromaDB client oluştur
chroma_client = chromadb.Client(Settings(
    anonymized_telemetry=False,
    allow_reset=True
))

# Koleksiyon oluştur (varsa sil)
try:
    chroma_client.delete_collection("istanbul_bolge_plani")
except:
    pass

collection = chroma_client.create_collection(
    name="istanbul_bolge_plani",
    metadata={"description": "2024-2028 İstanbul Bölge Planı RAG Database"}
)

print("✅ ChromaDB koleksiyonu oluşturuldu!")


💾 Vector database oluşturuluyor...
✅ ChromaDB koleksiyonu oluşturuldu!


In [14]:
# ============================================================================
# BÖLÜM 7: CHUNK'LARI VECTOR DB'YE EKLEME
# ============================================================================

print("\n⬆️ Chunk'lar vector database'e ekleniyor...")

batch_size = 10
for i in tqdm(range(0, len(chunks), batch_size), desc="Batch'ler ekleniyor"):
    batch_chunks = chunks[i:i+batch_size]
    
    # Her chunk için embedding oluştur
    embeddings = [embedding_model.embed_query(chunk) for chunk in batch_chunks]
    
    # ChromaDB'ye ekle
    collection.add(
        embeddings=embeddings,
        documents=batch_chunks,
        ids=[f"chunk_{j}" for j in range(i, i+len(batch_chunks))]
    )

print(f"✅ Toplam {len(chunks)} chunk eklendi!")
print(f"📊 Koleksiyon boyutu: {collection.count()}")


⬆️ Chunk'lar vector database'e ekleniyor...


Batch'ler ekleniyor: 100%|██████████| 88/88 [01:26<00:00,  1.02it/s]

✅ Toplam 875 chunk eklendi!
📊 Koleksiyon boyutu: 875


In [15]:
# ============================================================================
# BÖLÜM 8: RAG PIPELINE - SORGULAMA FONKSİYONU
# ============================================================================

print("\n🔍 RAG Pipeline oluşturuluyor...")

def retrieve_relevant_chunks(query, n_results=5):
    """Sorguya en yakın chunk'ları getirir"""
    
    # Query için embedding oluştur
    query_embedding = embedding_model.embed_query(query)
    
    # ChromaDB'den en yakın chunk'ları al
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )
    
    return results['documents'][0], results['ids'][0]

def generate_answer_with_gemini(query, context_chunks):
    """Gemini API ile cevap üretir"""
    
    # Context'i birleştir
    context = "\n\n".join(context_chunks)
    
    # Prompt oluştur
    prompt = f"""Sen İstanbul Bölge Planı uzmanı bir asistansın. Aşağıdaki belgeden alınan bilgilere dayanarak soruyu cevapla.

BELGE İÇERİĞİ:
{context}

SORU: {query}

CEVAP: Lütfen soruyu belge içeriğine dayanarak detaylı bir şekilde cevapla. Eğer bilgi belgede yoksa, bunu belirt."""
    
    try:
        # Gemini modeli oluştur
        model = genai.GenerativeModel('gemini-1.5-flash')
        
        # Cevap üret
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"⚠️ Gemini API hatası: {e}")
        return generate_answer_local(query, context_chunks)

def generate_answer_local(query, context_chunks):
    """Yerel (basit) cevap üretir - API hatası durumunda"""
    
    # Context'i birleştir
    context = "\n\n".join(context_chunks)
    
    # Basit özetleme (ilk 500 karakter + query vurgulama)
    answer = f"""İstanbul Bölge Planı belgesinde bulunan ilgili bilgiler:

{context[:800]}...

📌 Not: Bu cevap belge içeriğinden otomatik olarak çıkarılmıştır. Daha detaylı analiz için Gemini API'yi kullanabilirsiniz."""
    
    return answer

def rag_query(query, n_results=5, use_api=True):
    """Tam RAG pipeline - soru sor, cevap al"""
    
    print(f"\n❓ Soru: {query}")
    print("🔍 İlgili belgeler aranıyor...")
    
    # 1. Retrieve: İlgili chunk'ları getir
    relevant_chunks, chunk_ids = retrieve_relevant_chunks(query, n_results)
    
    print(f"✅ {len(relevant_chunks)} ilgili belge bulundu")
    print(f"📋 Chunk ID'ler: {chunk_ids}")
    
    # 2. Generate: Cevap üret
    print("🤖 Cevap oluşturuluyor...")
    
    if use_api:
        answer = generate_answer_with_gemini(query, relevant_chunks)
    else:
        answer = generate_answer_local(query, relevant_chunks)
    
    print(f"\n💬 CEVAP:\n{answer}")
    
    # İlgili chunk'ları da göster
    print(f"\n📚 KAYNAK BELGELER:")
    for i, (chunk_id, chunk) in enumerate(zip(chunk_ids, relevant_chunks), 1):
        print(f"\n[{i}] {chunk_id}")
        print(f"{chunk[:200]}...")
    
    return {
        "query": query,
        "answer": answer,
        "sources": chunk_ids,
        "relevant_chunks": relevant_chunks
    }

print("✅ RAG Pipeline hazır!")
print("💡 İpucu: use_api=False parametresi ile API olmadan çalışabilirsiniz")


🔍 RAG Pipeline oluşturuluyor...
✅ RAG Pipeline hazır!
💡 İpucu: use_api=False parametresi ile API olmadan çalışabilirsiniz


In [16]:
# ============================================================================
# BÖLÜM 9: TEST SORULARI
# ============================================================================

print("\n" + "="*80)
print("🧪 TEST SORULARI")
print("="*80)

# Test Soruları
test_queries = [
    "İstanbul bölge planının ana hedefleri nelerdir?",
    "Ulaşım altyapısı için neler planlanıyor?",
    "Yeşil alanlar ve çevre politikaları nelerdir?",
]

results = []

for query in test_queries:
    result = rag_query(query, n_results=3)
    results.append(result)
    print("\n" + "-"*80 + "\n")


🧪 TEST SORULARI

❓ Soru: İstanbul bölge planının ana hedefleri nelerdir?
🔍 İlgili belgeler aranıyor...
✅ 3 ilgili belge bulundu
📋 Chunk ID'ler: ['chunk_147', 'chunk_279', 'chunk_90']
🤖 Cevap oluşturuluyor...
⚠️ Gemini API hatası: 400 API Key not found. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API Key not found. Please pass a valid API key."
]

💬 CEVAP:
İstanbul Bölge Planı belgesinde bulunan ilgili bilgiler:

herkes için yüksek yaşam kalitesi sunan ,  
dirençli  İstanbul  
  45 Şekil 3. 2024 -2028 İstanbul Bölge Planı Vizyon Kavramsal Çerçevesi  
 
 
 
2024 -2028 İstanbul Bölge Planı Vizyonunu destekleyen 5 stratejik öncelik ve 3 yatay kesen öncelik 
belirlenmiştir.  
 
Stratejik Öncelikler;  
1. Teknolojik Dönüşüm: İstanbul’da yaşamın ve tüm sektörlerin teknolojik dönüşümünün 
hızlandırılması  
2. Yeşil Dönüşüm: İstanbul’un yeşil ekonomiye geç

In [17]:
# ============================================================================
# BÖLÜM 10: SONUÇLARI KAYDETME
# ============================================================================

print("\n💾 Sonuçlar kaydediliyor...")

# Sonuçları JSON olarak kaydet
with open("rag_results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("✅ Sonuçlar 'rag_results.json' dosyasına kaydedildi!")


💾 Sonuçlar kaydediliyor...
✅ Sonuçlar 'rag_results.json' dosyasına kaydedildi!


In [18]:
# ============================================================================
# BÖLÜM 11: İNTERAKTİF SORGULAMA
# ============================================================================

print("\n" + "="*80)
print("💬 İNTERAKTİF SORGULAMA MODU")
print("="*80)
print("Çıkmak için 'quit' yazın\n")

while True:
    user_query = input("🙋 Sorunuz: ")
    
    if user_query.lower() in ['quit', 'exit', 'çık', 'q']:
        print("👋 Görüşmek üzere!")
        break
    
    if user_query.strip():
        rag_query(user_query, n_results=3)
    print("\n" + "-"*80 + "\n")


💬 İNTERAKTİF SORGULAMA MODU
Çıkmak için 'quit' yazın

👋 Görüşmek üzere!


In [19]:
# ============================================================================
# BÖLÜM 12: İSTATİSTİKLER VE ÖZETİ
# ============================================================================

print("\n" + "="*80)
print("📊 PROJE İSTATİSTİKLERİ")
print("="*80)

stats = {
    "PDF Bilgileri": {
        "Dosya": PDF_PATH,
        "Toplam Karakter": f"{len(raw_text):,}",
        "Toplam Kelime": f"{len(raw_text.split()):,}"
    },
    "Chunking": {
        "Chunk Sayısı": len(chunks),
        "Chunk Boyutu": "1000 karakter",
        "Overlap": "200 karakter"
    },
    "Vector Database": {
        "Sistem": "ChromaDB",
        "Koleksiyon": "istanbul_bolge_plani",
        "Kayıt Sayısı": collection.count()
    },
    "Embedding": {
        "Model": "Google embedding-001",
        "Boyut": len(test_embedding)
    },
    "Generation": {
        "Model": "Gemini 1.5 Flash"
    }
}

for category, info in stats.items():
    print(f"\n{category}:")
    for key, value in info.items():
        print(f"  • {key}: {value}")

print("\n" + "="*80)
print("✅ NOTEBOOK TAMAMLANDI!")
print("="*80)


📊 PROJE İSTATİSTİKLERİ

PDF Bilgileri:
  • Dosya: ../Data/2024-2028-İstanbul-bölge-planı-taslak.pdf
  • Toplam Karakter: 714,440
  • Toplam Kelime: 86,220

Chunking:
  • Chunk Sayısı: 875
  • Chunk Boyutu: 1000 karakter
  • Overlap: 200 karakter

Vector Database:
  • Sistem: ChromaDB
  • Koleksiyon: istanbul_bolge_plani
  • Kayıt Sayısı: 875

Embedding:
  • Model: Google embedding-001
  • Boyut: 384

Generation:
  • Model: Gemini 1.5 Flash

✅ NOTEBOOK TAMAMLANDI!
